In [102]:
import numpy as np
import random
from PIL import Image
import torch
import os
import torchvision.transforms as transforms
from skimage.measure import compare_psnr

In [23]:
inp_dir = './output_images/sharp/'
iter1_dir = './output_images/iter_1/'
iter9_dir = './output_images/iter_9/'
blur_dir = './output_images/blur/'

inp_ims = os.listdir(inp_dir)
iter1_ims = os.listdir(iter1_dir)
iter9_ims = os.listdir(iter9_dir)
blur_ims = os.listdir(blur_dir)

In [27]:
f_inp, f_iter1, f_iter9, f_blur = list(), list(), list(), list()
for f in inp_ims:
    base_name = '_'.join(f.split('_')[1:])
    i1_found, i9_found, blur_found = False, False, False
    for fi1 in iter1_ims:
        if base_name in fi1:
            i1_found = True
            break
    for fi9 in iter9_ims:
        if base_name in fi9:
            i9_found = True
            break
    for fb in blur_ims:
        if base_name in fb:
            blur_found = True
            break
    if i1_found and i9_found:
        f_inp.append(f)
        f_iter1.append(fi1)
        f_iter9.append(fi9)
        f_blur.append(fb)
        
print(len(f_inp))

10


In [138]:
def inp_transform(fname, show=False, rotate=False):
    image = Image.open(fname)  #because one sharp image for multiple training images
    if show:
        image.show()
    if rotate:
        image = image.rotate(270)
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
        if show:
            image.show()
    image = np.array(image)
    print(image.shape)
    image = image[:, :, :3] # 4th channel is transparency... cut it out
    image = transforms.ToTensor()(image)
        
    return image

In [85]:
def loss_test1(sharp, blur):
    num_examples, channels, width, height = 1, 3, 256, 256
    assert(sharp.size() == blur.size())
    norm_factor = 1./(num_examples*channels*width*height)
    s = blur-sharp
    n = torch.norm(s)
    print(n)
    m = torch.sum(n)
    
    return norm_factor*m


def loss_test2(sharp, blur):
    num_examples, channels, width, height = 10, 3, 256, 256
    assert(sharp.size() == blur.size())
    norm_factor = 1./(num_examples*channels*width*height)
    s = (blur-sharp)**2
#     n = torch.norm(s)
#     print(n)
    m = torch.sum(s)
    
    return norm_factor*m

In [81]:
s = inp_transform(inp_dir + f_inp[index], show=True)
a = inp_transform(iter1_dir + f_iter1[index], show=True, rotate=True)

In [139]:
index = 1
print(f_inp[index])
print(f_iter1[index])

sharp_ims, b1_ims, b9_ims, blur_ims = list(), list(), list(), list()
for index in range(len(f_inp)):
    sharp_ims.append(inp_transform(inp_dir + f_inp[index]))
    b1_ims.append(inp_transform(iter1_dir + f_iter1[index], rotate=True))
    b9_ims.append(inp_transform(iter9_dir + f_iter9[index], rotate=True))
    blur_ims.append(inp_transform(blur_dir + f_blur[index]))
sharp_ims = torch.stack(sharp_ims)
b1_ims = torch.stack(b1_ims)
b9_ims = torch.stack(b9_ims)
blur_ims = torch.stack(blur_ims)

print(blur_ims.size())


sharp_c_n07881800_12870.JPEG
out_ws4_c_n07881800_12870.JPEG
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
(256, 256, 4)
(256, 256, 3)
(256, 256, 3)
(256, 3840, 3)
torch.Size([10, 3, 256, 3840])


In [111]:
sharp_im.size()

torch.Size([3, 256, 256])

In [112]:
b1_im.size()

torch.Size([3, 256, 256])

In [113]:
loss_test2(sharp_ims, b1_ims)

0.056699238576142244

In [114]:
loss_test2(sharp_ims, b9_ims)

0.0531722666494911

In [115]:
select = 4
loss_test2(sharp_ims, blur_ims[:, :, :, select*256:(select+1)*256])

0.047725080180383725

In [94]:
input_image = Image.open(blur_dir + f_blur[index])
input_image = np.array(input_image)

input_image = np.reshape(input_image, (256, 256, 15, 3), order='F')
show_from_numpy(input_image[:, :, 6])
input_image = input_image[:, :, 3:13, :]
input_image = np.swapaxes(input_image, 2, 3)
input_image = np.reshape(input_image, (256, 256, 30), order='F')
# input_image.show(input_image[:, :, :3])
input_image = transforms.ToTensor()(input_image)
demo_image = transforms.ToPILImage()(input_image)
demo_image.show()




In [92]:
def show_from_numpy(demo_image):
    demo_image = transforms.ToTensor()(demo_image)
    demo_image = transforms.ToPILImage()(demo_image)
    demo_image.show()

In [101]:
input_image = Image.open(blur_dir + f_blur[index])
input_image = np.array(input_image)
input_image = np.reshape(input_image, (256, 256, 15, 3), order='F')
show_from_numpy(input_image[:, :, 7])
input_image = np.reshape(input_image, (256, 256, 45), order='C')
show_from_numpy(input_image[:, :, 12:15])

In [124]:
def compute_psnr(sharp, blur):
    size = 10
    for i in range(size):
        sharp_im = transforms.To
    return torch.sum([compare_psnr(transforms.ToPILImage()(sharp[i]), transforms.ToPILImage()(blur[i])) for i in range(size)])
#     return compare_psnr(sharp, blur)

In [125]:
compute_psnr(sharp_ims, b1_ims)

AttributeError: 'Image' object has no attribute 'dtype'

In [137]:
s = sharp_ims.numpy()
b = blur_ims.numpy()

print(s.shape)
s0 = s[0]
# compute_psnr(s[0], b[0])


(10, 3, 256, 256)
